# Amalie's Birthday
## The Three Codeteers Dashboard

### Import libraries

In [22]:
import pandas as pd
import emoji
import re
import numpy as np
from stop_words import get_stop_words 
from ftfy import fix_text
from json import load

### Load messages

In [91]:
with open('message.json') as json_file:  # Load train data
    dict_ = load(json_file)
    
dict_msg = dict_["messages"]
#dict_msg = dict_msg[0:200]

### Emoji dictionary

In [92]:
# Create a dictionary of symbols and codes of emoji
emoji_keys = [':-)',':-d',':-3',':-p',':-o',':-0',
                 '=-)','=-d','=-3','=-d','=-o','=-0',
                 'x-d',';-d',';-)','>.<', 'x.x','<3',
                 ':)',':d',':3',':p',':o',':0',
                 '=)','=d','=3','=p','=o','=0',
                 'xd',';d',';)',':(', ':-(','=(', 
                 '=-(',":'("]

emoji_values = [':simple_smile:',':smiley:',':smiley_cat:',':stuck_out_tongue:',':open_mouth:',':open_mouth:',
                ':simple_smile:',':smiley:',':smiley_cat:',':stuck_out_tongue:',':open_mouth:',':open_mouth:',
                 ':laughing:',':wink:',':wink:',':persevere:',':dizzy_face:',':heart:',
                 ':simple_smile:',':smiley:',':smiley_cat:',':stuck_out_tongue:',':open_mouth:',':open_mouth:',
                 ':simple_smile:',':smiley:',':smiley_cat:',':stuck_out_tongue:',':open_mouth:',':open_mouth:',
                 ':laughing:',':wink:',':sad:',':pensive:',':pensive:',':pensive:',
                 ':pensive:',':cry:']

dict_emoji = dict(zip(emoji_keys, emoji_values))

### Clean messages and store in a list

In [96]:
clean_list = []
number_words = []

for message in dict_msg: # iterate over dictionaries of recipes
    
    # If there is a content key...
    if 'content' in message.keys():
        # Look in the content key
        clean_msg = fix_text(message['content'])
        
        # Remove URLs
        clean_msg = re.sub(r"http\S+", "", clean_msg)
        
        # Convert letters to lower case
        clean_msg = clean_msg.lower()
        
        # Get emoji in Meeke's format from the message
        list_emoji_keys = re.findall('(:-*\)|:-*\(|=-*\(|:-*D|:-*3|:-*P|:-*o|:-*0|=-*\)|=-*D|=-*3|'
                          '=-*P|=-*o|=-*0|x-*D|;-*D|;-*\)|\>\.\<|x.x|\<3)', 
                          clean_msg, re.IGNORECASE)
        
        # Replace original emoji with readable format
        if list_emoji_keys != []:            
            for emoji_ in list_emoji_keys:
                clean_msg = clean_msg.replace(emoji_," "+dict_emoji[emoji_])
                
        for char in clean_msg:            
            # Remove special characters (excluding emoji, spaces : or _, which appear in the emoji)
            if not(char.islower() or char in [' ','_',':',"'"] or char in emoji.UNICODE_EMOJI): 
                clean_msg = clean_msg.replace(char, "")
            
            # If emoji, add a space before the emoji
            if char in emoji.UNICODE_EMOJI:
                clean_msg  = emoji.demojize(clean_msg.replace(char,' ' + char))
        
        # Create a list of words
        number_words.append(len(clean_msg.split()))
        clean_list.append(clean_msg.split())    
        
        #print(clean_msg.split())
clean_list[0:10]

[["it's", 'exercise', 'time'],
 ['sounds', 'good', 'with', 'a', 'walk', ':grinning_face:'],
 ['and', 'you', 'can', 'tell', 'us', 'everything', 'about', 'the', 'wedding'],
 ['but', 'i', 'can', 'join', 'you', 'for', 'the', 'post', 'lunch', 'walk'],
 ['i', 'have', 'leftovers', 'to', 'finish', ':face_with_hand_over_mouth:'],
 ['cool',
  'do',
  'you',
  'want',
  'to',
  'have',
  'lunch',
  'break',
  'together',
  ':simple_smile:'],
 ['yes'],
 ['are', 'you', 'also', 'sitting', 'in', 'the', 'library', ':simple_smile:'],
 ['haha', 'no'],
 [':grinning_face_with_smiling_eyes:']]

### Create dataframe - message level

In [101]:
# Convert list of dictionaries to dataframe
df_msg = pd.DataFrame(dict_msg)

# Filter rows of messages with content
content_true = [isinstance(elem, str) for elem in df_msg['content']]
df_msg = df_msg[content_true].reset_index()

# Take only names and time
df_msg = df_msg[['sender_name', 'timestamp']]

# Fix bad unicode characters in sender and change time format
df_msg['timestamp']   = df_msg['timestamp'].apply(datetime.datetime.fromtimestamp)
df_msg['sender_name'] = df_msg['sender_name'].apply(fix_text)
df_msg['number_words'] = number_words
df_msg.head(10)

,sender_name,timestamp,number_words
0,Paula Fermín Cueto,2018-05-09 13:57:22,3
1,Amalie Pauli Nyhagen,2018-05-09 08:39:14,6
2,Paula Fermín Cueto,2018-05-09 08:38:39,9
3,Paula Fermín Cueto,2018-05-09 08:38:13,10
4,Paula Fermín Cueto,2018-05-09 08:38:11,6
5,Amalie Pauli Nyhagen,2018-05-09 08:37:42,10
6,Paula Fermín Cueto,2018-05-09 08:36:57,1
7,Amalie Pauli Nyhagen,2018-05-09 08:36:25,8
8,Amalie Pauli Nyhagen,2018-05-09 08:05:33,2
9,Meeke Roet,2018-05-09 08:04:15,1


### Dataframe - word level

In [102]:
# Repeat row as many times as words in the the message
df_msg = df_msg.set_index(['sender_name'])['timestamp'].repeat(df_msg['number_words']).reset_index()
df_msg.head(10)

,sender_name,timestamp
0,Paula Fermín Cueto,2018-05-09 13:57:22
1,Paula Fermín Cueto,2018-05-09 13:57:22
2,Paula Fermín Cueto,2018-05-09 13:57:22
3,Amalie Pauli Nyhagen,2018-05-09 08:39:14
4,Amalie Pauli Nyhagen,2018-05-09 08:39:14
5,Amalie Pauli Nyhagen,2018-05-09 08:39:14
6,Amalie Pauli Nyhagen,2018-05-09 08:39:14
7,Amalie Pauli Nyhagen,2018-05-09 08:39:14
8,Amalie Pauli Nyhagen,2018-05-09 08:39:14
9,Paula Fermín Cueto,2018-05-09 08:38:39


Maybe it would be a good idea to add a column 'type' where we specify if a word is a stop word or an emoji, would make things easier for Tableau